In [17]:
import tensorflow as tf
import pandas as pd
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import matplotlib.image as img
import cv2
import numpy as np
import os

In [18]:
def generate_dataset(animals):
    
    base_dir = 'Data/animals/'
    df = pd.DataFrame(columns=['image', 'animal'])
    
    for animal in animals:
        directory = base_dir + animal
        try:
            for filename in os.listdir(directory):
                f = os.path.join(directory, filename)
                if os.path.isfile(f):
                    df = df.append({'image': f, 'animal': animal}, ignore_index=True)
        except:
            print(animal + ' Does not exist')
            return None
        
    return df

In [19]:
def create_image_matrix(path):
    image = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    image = cv2.resize(image, (96, 96))
    #image = np.array(image).astype('float32')
    #plt.imshow(image)
    return image

In [20]:
create_image_matrix('Data/animals/antelope/0a37838e99.jpg')

array([[[ 86, 126, 144],
        [ 94, 127, 147],
        [ 89, 123, 139],
        ...,
        [111, 148, 168],
        [111, 147, 167],
        [109, 144, 165]],

       [[ 80, 123, 138],
        [ 88, 127, 140],
        [ 88, 124, 139],
        ...,
        [118, 155, 175],
        [114, 151, 171],
        [110, 149, 169]],

       [[ 74, 119, 132],
        [ 78, 124, 135],
        [ 86, 126, 131],
        ...,
        [122, 156, 179],
        [113, 153, 172],
        [112, 150, 171]],

       ...,

       [[ 74, 115, 146],
        [106, 147, 161],
        [ 98, 140, 159],
        ...,
        [ 97, 141, 168],
        [ 86, 141, 163],
        [ 72, 134, 154]],

       [[ 97, 137, 170],
        [ 98, 138, 157],
        [152, 185, 194],
        ...,
        [ 95, 141, 169],
        [ 67, 114, 140],
        [ 67, 129, 148]],

       [[ 77, 117, 145],
        [ 87, 128, 150],
        [102, 145, 152],
        ...,
        [ 76, 104, 130],
        [ 75, 124, 144],
        [ 69, 135, 155]]

In [21]:
from sklearn.model_selection import train_test_split

df = generate_dataset(['bee', 'cat', 'bison', 'bear'])

df['image'] = df['image'].apply(create_image_matrix)

df_train_x, df_test_x, df_train_y, df_test_y = train_test_split(df['image'], df['animal'], test_size=0.3)

df_train_y = pd.get_dummies(df_train_y)
df_test_y = pd.get_dummies(df_test_y)

In [22]:
df_train_x = np.asarray(df_train_x)
df_train_x[0].shape

(96, 96, 3)

In [23]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(96, 96, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(60, activation='softmax'))

In [24]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(df_train_x, df_train_y, epochs=1)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).